In [1]:
# notebook settings
%load_ext autoreload
%autoreload 2

# external imports
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from copy import copy
from math import floor

# internal imports
from pympc.geometry.polyhedron import Polyhedron
from pympc.dynamics.discrete_time_systems import LinearSystem, AffineSystem, PieceWiseAffineSystem
from pympc.control.hybrid_benchmark.controllers import HybridModelPredictiveController
from pympc.plot import plot_input_sequence, plot_state_trajectory, plot_output_trajectory
from pympc.control.hybrid_benchmark.utils import get_constraint_set, remove_redundant_inequalities_fast, convex_hull_method_fast

# Problem set-up

In [9]:
# numeric parameters of the system
m = 1.
r = .1
I = .4*m*r**2.
d = .5
k = 500.
bt = 20.
bn = 10.
mu = .2
g = 10.
h = .025

In [10]:
# state: x = (qxb, qyb, tzb, qxp, qyp, vxb, vyb, ozb)
# input: u = (vxp, vyp)

# discretization method
method = 'zero_order_hold'

# dynamics n.1 (ball in the air)
A1 = np.block([
    [np.zeros((3,5)), np.eye(3)],
    [np.zeros((5,8))]
    ])
B1 = np.block([
    [np.zeros((3,2))],
    [np.eye(2)],
    [np.zeros((3,2))]
    ])
c1 = np.array([0.]*6 + [-g, 0.])
S1 = AffineSystem.from_continuous(A1, B1, c1, h, method)

# dynamics n.2 (paddle pulling ball)
S2 = copy(S1)

# dynamics n.3 (paddle pushing ball, sticking)
# (no offset terms, the zero of the position of the ball is shifted by m*g/k-r)
A3 = copy(A1)
A3[5,5] = -bt/m
A3[5,7] = -r*bt/m
A3[6,1] = -k/m
A3[6,4] = k/m
A3[6,6] = -bn/m
A3[7,5] = -r*bt/I
A3[7,7] = -r*r*bt/I
B3 = copy(B1)
B3[5,0] = bt/m
B3[6,1] = bn/m
B3[7,0] = r*bt/I
S3 = LinearSystem.from_continuous(A3, B3, h, method)

# dynamics n.4 (paddle pushing ball, ball sliding left wrt paddle)
A4 = copy(A3)
A4[5] = mu * A4[6]
A4[7] = r*m/I * A4[5]
B4 = copy(B3)
B4[5] = mu * B4[6]
B4[7] = r*m/I * B4[5]
c4 = np.array([0.]*5 + [mu*g, 0., r*m*mu*g/I])
S4 = AffineSystem.from_continuous(A4, B4, c4, h, method)

# dynamics n.5 (paddle pushing ball, ball sliding right wrt paddle)
A5 = copy(A4)
A5[5] = -A5[5]
A5[7] = -A5[7]
B5 = copy(B4)
B5[5] = -B5[5]
B5[7] = -B5[7]
c5 = -c4
S5 = AffineSystem.from_continuous(A5, B5, c5, h, method)

# list of dynamics
S_list = [S1, S2, S3, S4, S5]

In [11]:
# state: x = (qxb, qyb, tzb, qxp, qyp, vxb, vyb, ozb)
# input: u = (vxp, vyp)

### limite sulla forza massima

# state bounds
x_max = np.array([1., 3., 3*np.pi, .5, 1., 10., 10., 20.])
x_min = - x_max # np.array([-1., -1., -1.2, -.2, -5., -5.])

# input bounds
u_max = np.array([100., 100.])*2.
u_min = - u_max

# domain bounds
xu_min = np.concatenate((x_min, u_min))
xu_max = np.concatenate((x_max, u_max))

'''
domain n.1, out of contact if:
qyb >= qyp + mg/k
'''
D1 = Polyhedron.from_bounds(xu_min, xu_max)
D1.add_inequality(
    np.array([[0., -1., 0., 0., 1., 0., 0., 0., 0., 0.]]),
    np.array([-m*g/k])
)
assert D1.bounded
assert not D1.empty

'''
domain n.2, pulling if:
- qyb <= qyp + mg/k,
- fn = k(qyp-qyb) + bn(vyp-vyb) + mg <= 0
'''
D2 = Polyhedron.from_bounds(xu_min, xu_max)
D2.add_inequality(
    np.array([[0., 1., 0., 0., -1., 0., 0., 0., 0., 0.]]),
    np.array([m*g/k])
)
D2.add_inequality(
    np.array([[0., -k, 0., 0., k, 0., -bn, 0., 0., bn]]),
    np.array([-m*g])
)
assert D2.bounded
assert not D2.empty

'''
domain n.3, sticking if:
- qyb <= qyp + mg/k,
- fn = k(qyp-qyb) + bn(vyp-vyb) + mg >= 0,
- ft = bt(vxp-vxb-r ozb) <= mu fn = mu k(qyp-qyb) + mu bn(vyp-vyb) + mu mg,
- ft = bt(vxp-vxb-r ozb) >= -mu fn = -mu k(qyp-qyb) - mu bn(vyp-vyb) - mu mg,
- qxb - qxp <= d,
- qxp - qxb <= d
'''
D3 = Polyhedron.from_bounds(xu_min, xu_max)
D3.add_inequality(
    np.array([[0., 1., 0., 0., -1., 0., 0., 0., 0., 0.]]),
    np.array([m*g/k])
)
D3.add_inequality(
    np.array([[0., k, 0., 0., -k, 0., bn, 0., 0., -bn]]),
    np.array([m*g])
)
D3.add_inequality(
    np.array([[0., mu*k, 0., 0., -mu*k, -bt, mu*bn, -r*bt, bt, -mu*bn]]),
    np.array([mu*m*g])
)
D3.add_inequality(
    np.array([[0., mu*k, 0., 0., -mu*k, bt, mu*bn, r*bt, -bt, -mu*bn]]),
    np.array([mu*m*g])
)
D3.add_inequality(
    np.array([[1., 0., 0., -1., 0., 0., 0., 0., 0., 0.]]),
    np.array([d])
)
D3.add_inequality(
    np.array([[-1., 0., 0., 1., 0., 0., 0., 0., 0., 0.]]),
    np.array([d])
)
assert D3.bounded
assert not D3.empty

'''
domain n.4, ball sliding left wrt paddle if:
- qyb <= qyp + mg/k,
- fn = k(qyp-qyb) + bn(vyp-vyb) + mg >= 0,
- ft = bt(vxp-vxb-r ozb) >= mu fn = mu k(qyp-qyb) + mu bn(vyp-vyb) + mu mg,
- qxb - qxp <= d,
- qxp - qxb <= d
'''
D4 = Polyhedron.from_bounds(xu_min, xu_max)
D4.add_inequality(
    np.array([[0., 1., 0., 0., -1., 0., 0., 0., 0., 0.]]),
    np.array([m*g/k])
)
D4.add_inequality(
    np.array([[0., k, 0., 0., -k, 0., bn, 0., 0., -bn]]),
    np.array([m*g])
)
D4.add_inequality(
    np.array([[0., -mu*k, 0., 0., mu*k, bt, -mu*bn, r*bt, -bt, mu*bn]]),
    np.array([-mu*m*g])
)
D3.add_inequality(
    np.array([[1., 0., 0., -1., 0., 0., 0., 0., 0., 0.]]),
    np.array([d])
)
D3.add_inequality(
    np.array([[-1., 0., 0., 1., 0., 0., 0., 0., 0., 0.]]),
    np.array([d])
)
assert D4.bounded
assert not D4.empty

'''
domain n.5, ball sliding right wrt paddle if:
- qyb <= qyp + mg/k,
- fn = k(qyp-qyb) + bn(vyp-vyb) + mg >= 0,
- ft = bt(vxp-vxb-r ozb) <= -mu fn = -mu k(qyp-qyb) - mu bn(vyp-vyb) - mu mg,
- qxb - qxp <= d,
- qxp - qxb <= d
'''
D5 = Polyhedron.from_bounds(xu_min, xu_max)
D5.add_inequality(
    np.array([[0., 1., 0., 0., -1., 0., 0., 0., 0., 0.]]),
    np.array([m*g/k])
)
D5.add_inequality(
    np.array([[0., k, 0., 0., -k, 0., bn, 0., 0., -bn]]),
    np.array([m*g])
)
D5.add_inequality(
    np.array([[0., -mu*k, 0., 0., mu*k, -bt, -mu*bn, -r*bt, bt, mu*bn]]),
    np.array([-mu*m*g])
)
D3.add_inequality(
    np.array([[1., 0., 0., -1., 0., 0., 0., 0., 0., 0.]]),
    np.array([d])
)
D3.add_inequality(
    np.array([[-1., 0., 0., 1., 0., 0., 0., 0., 0., 0.]]),
    np.array([d])
)
assert D5.bounded
assert not D5.empty

# list of domains
D_list = [D1, D2, D3, D4, D5]

In [32]:
# state: x = (qxb, qyb, tzb, qxp, qyp, vxb, vyb, ozb)
# input: u = (vxp, vyp)

# PWA system
S = PieceWiseAffineSystem(S_list, D_list)

# controller parameters
N = 100
Q = np.diag([1.,1.,1.,1.,1.,1.,1.,1.])
R = np.diag([1.,1.])

# terminal set and cost
P = Q
xN_min = np.array([-1.,0.,0.,0.,0.,0.,0.,0.])
xN_max = np.array([1.,0.,0.,0.,0.,0.,0.,0.])
# X_N = Polyhedron.from_bounds(*[np.array([0.0,0.,0.,0.,0.,0.,0.,0.])]*2)
X_N = Polyhedron.from_bounds(xN_min, xN_max)

In [33]:
controller = HybridModelPredictiveController(S, N, Q, R, P, X_N, method='Convex hull')
# controller.prog.setParam('Heuristics', 0)

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


In [34]:
# initial condition
# x0 = np.array([0.,0.,2.*np.pi,0.,0.,0.,0.,0.])
x0 = np.array([0.,0.,0.1,0.,0.,0.,0.,0.])

In [ ]:
u, x, ms, cost = controller.feedforward(x0)

Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Optimize a model with 21880 rows, 10508 columns and 74290 nonzeros
Model has 1008 quadratic objective terms
Variable types: 10008 continuous, 500 integer (500 binary)
Coefficient statistics:
  Matrix range     [4e-04, 5e+02]
  Objective range  [0e+00, 0e+00]
  QObjective range [1e+00, 1e+00]
  Bounds range     [1e-01, 1e-01]
  RHS range        [1e+00, 1e+00]
Presolve removed 8664 rows and 4093 columns
Presolve time: 0.16s
Presolved: 13216 rows, 6415 columns, 47082 nonzeros
Presolved model has 993 quadratic objective terms
Variable types: 5917 continuous, 498 integer (498 binary)

Root relaxation: objective 1.293107e-02, 24437 iterations, 3.15 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.01293    0  365          -    0.01293      -     -    3s
     0     0    0.01293    0  365          

  1824  1466     cutoff  202         1.75976    0.01911  98.9%   660  721s
  1885  1487    0.05637   27  404    1.75976    0.01911  98.9%   651  730s
  1929  1516    0.05641   34  391    1.75976    0.01911  98.9%   644  741s
  2012  1567    0.10799   51  370    1.75976    0.01911  98.9%   629  750s


In [27]:
plot_input_sequence(u, h, (u_min, u_max))

TypeError: 'NoneType' object has no attribute '__getitem__'

In [28]:
plot_state_trajectory(x, h, (x_min, x_max))

TypeError: 'NoneType' object has no attribute '__getitem__'

In [29]:
# x0 = np.array([0.,0.,0.,0.,0.,.0,0.,0.])
u = [np.array([0.,0.])]*3 + [np.array([0.,0.])]*500
x, ms = S.simulate(x0, u)
print ms

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

# Animation

In [30]:
import meshcat
from meshcat.geometry import Box, Sphere, Cylinder, MeshLambertMaterial
from meshcat.animation import Animation
import meshcat.transformations as tf

In [20]:
vis = meshcat.Visualizer()
# vis.jupyter_cell()
vis.open()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


<Visualizer using: <meshcat.visualizer.ViewerWindow instance at 0x10921dc20> at path: <meshcat.path.Path object at 0x1092b7f68>>

In [21]:
paddle_z = .05
paddle_x = .3
red = 0xff2222
blue = 0x2222ff
green = 0x22ff22
vis['paddle'].set_object(Box([paddle_x, d*2., paddle_z]), MeshLambertMaterial(color=red))
vis['ball'].set_object(Sphere(r), MeshLambertMaterial(color=blue))
vis['ball_orientation'].set_object(Cylinder(.01, r), MeshLambertMaterial(color=green))

In [31]:
anim = Animation()
for t, xt in enumerate(x):
    with anim.at_frame(vis, t*h*30) as frame:
        frame['paddle'].set_transform(
            tf.translation_matrix([0, xt[3], xt[4]-paddle_z/2.])
        )
        frame['ball'].set_transform(
            tf.translation_matrix([0, xt[0], xt[1]+r-m*g/k])
        )
        frame['ball_orientation'].set_transform(
            tf.translation_matrix([0, xt[0], xt[1]+r-m*g/k]).dot(
                tf.rotation_matrix(xt[2], [1.,0.,0.])
            )
        )
vis.set_animation(anim)